# Assignment 2
## by Vinit Saah

#### Initialize base overlay and import libraries

In [1]:
import random
import threading
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pynq.lib.rgbled as rgbled 
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

### blink LED. Based on time (t), counter value is calculated. blink rate determines rate of blinking LED

In [2]:
def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: time to blink the LED for a given task
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 3)
    '''
    t = round(t/d) #converting the time into counter value
    for i in range(t):
        if 0 <= n and n< 4:
            base.leds[n].toggle()
            time.sleep(d)
        elif n == 4:
            rgbled.RGBLED(n).write(0x2) #Green color is 0x02
            time.sleep(d)
            rgbled.RGBLED(n).write(0x0)
            time.sleep(d)
        else:
            print("Invalid LED IDs to blink")
            return
    if 0 <= n and n < 4:
        base.leds[n].off()

### Test code to blink LED

In [ ]:
blink(1, 0.1, 0)
blink(2,0.02,4)
blink(2,0.02,7)

### Let all diners be righty except 4th diner with ID 0. Let him be lefty. This way, atleast in worst case possible, 5th diner would get the 

In [7]:
def bp_monitor(button_lock, btns):
    global stop_dining
    while stop_dining == False:
        time.sleep(0.01)
        if btns.read() != 0:
            button_lock.acquire(True)
            stop_dining = True
            #print("Button {} is pressed stop_dining = \n", stop_dining)
            button_lock.release()
        


def dining_philosopher(f_right, f_left, button_lock, id_diner):   
    global stop_dining, data, mutex_fork
    EATING = 1
    NAPPING = 2
    STARVING = 3
    
    count = 10
    
    diner_state = STARVING 
    
    eating_count = 0
    napping_count = 0
    starving_count = 0
    
    halt = False
    
    start_time = start_starving_time = time.time() #starving time 1, later to be reset after napping

    while halt == False:
        f_left_available = False
        f_right_available = False
        #take a lock for shared list
        b_mutex_avail = mutex_fork.acquire(False)
        #if id is 3 first take left fork, others take right fork
        if b_mutex_avail:
            if id_diner == 3:
                f_left_available = f_left.acquire(True)
                f_right_available = f_right.acquire(False)
            else:
                f_right_available = f_right.acquire(True)
                f_left_available = f_left.acquire(False)
        
            if f_right_available and f_left_available:
                diner_state = EATING
                end_starving_time = time.time()
                data['starving time']['Phil {}'.format(id_diner)] += end_starving_time-start_starving_time
            else:
                if f_right_available:
                    f_right.release()
                if f_left_available:
                    f_left.release()
            mutex_fork.release()
        
        if diner_state == EATING:
            eating_count += 1 
            #print("Diner {} is EATING\n".format(id_diner))
            data['eat count']['Phil {}'.format(id_diner)] += 1
            start_eating_time = time.time()
            blink(1, 0.02,id_diner)
            f_right.release()
            f_left.release()
            diner_state = NAPPING
            end_eating_time = time.time()
            data['eating time']['Phil {}'.format(id_diner)] += end_eating_time - start_eating_time
            #time.sleep(0.1) #Yield  
    
        if diner_state == NAPPING:
            #print("Diner {} is NAPPING\n".format(id_diner))
            napping_count +=1
            data['nap count']['Phil {}'.format(id_diner)] += 1
            start_napping_time = time.time()
            blink(0.5, 0.1,id_diner)
            end_napping_time = time.time()
            data['napping time']['Phil {}'.format(id_diner)] += end_napping_time-start_napping_time
            diner_state = STARVING
            start_starving_time = time.time()
            #time.sleep(0.1) #Yield
    
        if diner_state == STARVING:
            starving_count += 1
            #print("Diner {} is STARVING\n".format(id_diner))
            data['starve count']['Phil {}'.format(id_diner)] += 1
            #time.sleep(0.1)

        button_lock.acquire(True)
        if stop_dining == True:
            #print("stop_dining is ", stop_dining)
            halt = True
        button_lock.release()
    
    data['time sum']['Phil {}'.format(id_diner)] = data['eating time']['Phil {}'.format(id_diner)] + \
                                              data['napping time']['Phil {}'.format(id_diner)] + \
                                              data['starving time']['Phil {}'.format(id_diner)]
    data['run time']['Phil {}'.format(id_diner)] = int(time.time())-start_time
        
    #print("Diner = ",id_diner, "eating count =", eating_count,"napping count =", napping_count, "starving_count =", starving_count)
        
stop_dining = False

#Data metrics
cols = ['run time', 'eat count', 'nap count', 'starve count', 'starving time',
      'eating time', 'napping time', 'time sum']
rows = ['Phil 0', 'Phil 1', 'Phil 2', 'Phil 3', 'Phil 4']
data = pd.DataFrame(np.zeros((5,8)),rows,cols)


fork_lock = list() #individual lock per philosopher
mutex_fork = threading.Lock() #fork_lock list would be accessible to each member, so need to protect it under critical section

if __name__ == "__main__":    
    
    philosophers_threads = list() #List of thread
    button_lock = threading.Lock() #Lock for getting button status, as philosophers reads it and other thread write to it.
    btns = base.btns_gpio
    
    for index in range(5):
        f_l = threading.Lock()
        fork_lock.append(f_l)

    b_t = threading.Thread(target = bp_monitor, args=(button_lock, btns))
    b_t.start()
    
    for i in range(5):    
        p_t = threading.Thread(target = dining_philosopher, args=(fork_lock[i], fork_lock[i-1],button_lock,i))
        philosophers_threads.append(p_t)
        p_t.start()
        
   #Button press thread would exit first
    name = b_t.getName()
    b_t.join()
    print('Button Press Monitor {} joined'.format(name))
    
    #Philosophers would then exit depending upon their activity
    for t in philosophers_threads:
        name = t.getName()
        t.join()
        print('Philosopher{} joined'.format(name))
    data = data.round(1)


Button Press Monitor Thread-16 joined
PhilosopherThread-17 joined
PhilosopherThread-18 joined
PhilosopherThread-19 joined
PhilosopherThread-20 joined
PhilosopherThread-21 joined


In [8]:
data

,run time,eat count,nap count,starve count,starving time,eating time,napping time,time sum
Phil 0,26.4,5.0,5.0,4974.0,13.6,7.7,2.6,23.9
Phil 1,26.4,7.0,7.0,5859.0,13.9,9.2,3.7,26.7
Phil 2,25.4,8.0,8.0,5189.0,10.1,11.4,4.2,25.6
Phil 3,25.4,3.0,3.0,8523.0,18.6,3.7,1.6,24.0
Phil 4,28.3,5.0,5.0,1987.0,10.0,13.2,5.2,28.3


In [9]:
def bp_monitor(button_lock, btns):
    global stop_dining
    while stop_dining == False:
        time.sleep(0.01)
        if btns.read() != 0:
            button_lock.acquire(True)
            stop_dining = True
            print("Button {} is pressed stop_dining = \n", stop_dining)
            button_lock.release()
        


def dining_philosopher(f_right, f_left, button_lock, id_diner):   
    global stop_dining, data, mutex_fork
    EATING = 1
    NAPPING = 2
    STARVING = 3
    
    count = 10
    
    diner_state = STARVING 
    
    eating_count = 0
    napping_count = 0
    starving_count = 0
    
    halt = False
    
    start_time = start_starving_time = time.time() #starving time 1, later to be reset after napping

    while halt == False:
        f_left_available = False
        f_right_available = False
        #take a lock for shared list
        b_mutex_avail = mutex_fork.acquire(False)
        #if id is 3 first take left fork, others take right fork
        if id_diner == 3:
            f_left_available = f_left.acquire(True)
            f_right_available = f_right.acquire(False)
        else:
            f_right_available = f_right.acquire(True)
            f_left_available = f_left.acquire(False)
        
        if f_right_available and f_left_available:
            diner_state = EATING
            end_starving_time = time.time()
            data['starving time']['Phil {}'.format(id_diner)] += end_starving_time-start_starving_time
        else:
            if f_right_available:
                f_right.release()
            if f_left_available:
                f_left.release()
        if b_mutex_avail:
            mutex_fork.release()
        
        if diner_state == EATING:
            eating_count += 1 
            #print("Diner {} is EATING\n".format(id_diner))
            data['eat count']['Phil {}'.format(id_diner)] += 1
            start_eating_time = time.time()
            rand_eat = random.randint(5,10)
            blink(rand_eat, 0.02,id_diner)
            f_right.release()
            f_left.release()
            diner_state = NAPPING
            end_eating_time = time.time()
            data['eating time']['Phil {}'.format(id_diner)] += end_eating_time - start_eating_time
            #time.sleep(0.1) #Yield  
    
        if diner_state == NAPPING:
            #print("Diner {} is NAPPING\n".format(id_diner))
            napping_count +=1
            data['nap count']['Phil {}'.format(id_diner)] += 1
            start_napping_time = time.time()
            rand_nap = random.randint(1,4)
            blink(rand_nap, 0.1,id_diner)
            end_napping_time = time.time()
            data['napping time']['Phil {}'.format(id_diner)] += end_napping_time-start_napping_time
            diner_state = STARVING
            start_starving_time = time.time()
            #time.sleep(0.1) #Yield
    
        if diner_state == STARVING:
            starving_count += 1
            #print("Diner {} is STARVING\n".format(id_diner))
            data['starve count']['Phil {}'.format(id_diner)] += 1
            time.sleep(0.1) #Yield

        button_lock.acquire(False)
        if stop_dining == True:
            #print("stop_dining is ", stop_dining)
            halt = True
        button_lock.release()
    
    data['time sum']['Phil {}'.format(id_diner)] = data['eating time']['Phil {}'.format(id_diner)] + \
                                              data['napping time']['Phil {}'.format(id_diner)] + \
                                              data['starving time']['Phil {}'.format(id_diner)]
    data['run time']['Phil {}'.format(id_diner)] = int(time.time())-start_time
        
    #print("Diner = ",id_diner, "eating count =", eating_count,"napping count =", napping_count, "starving_count =", starving_count)

stop_dining = False

#Data metrics
cols = ['run time', 'eat count', 'nap count', 'starve count', 'starving time',
      'eating time', 'napping time', 'time sum']
rows = ['Phil 0', 'Phil 1', 'Phil 2', 'Phil 3', 'Phil 4']
data = pd.DataFrame(np.zeros((5,8)),rows,cols)


fork_lock = list() #individual lock per philosopher
mutex_fork = threading.Lock() #fork_lock list would be accessible to each member, so need to protect it under critical section

if __name__ == "__main__":    
    
    philosophers_threads = list() #List of thread
    button_lock = threading.Lock() #Lock for getting button status, as philosophers reads it and other thread write to it.
    btns = base.btns_gpio
    
    for index in range(5):
        f_l = threading.Lock()
        fork_lock.append(f_l)

    b_t = threading.Thread(target = bp_monitor, args=(button_lock, btns))
    b_t.start()
    
    for i in range(5):    
        p_t = threading.Thread(target = dining_philosopher, args=(fork_lock[i], fork_lock[i-1],button_lock,i))
        philosophers_threads.append(p_t)
        p_t.start()
        
   #Button press thread would exit first
    name = b_t.getName()
    b_t.join()
    print('Button Press Monitor {} joined'.format(name))
    
    #Philosophers would then exit depending upon their activity
    for t in philosophers_threads:
        name = t.getName()
        t.join()
        print('Philosopher{} joined'.format(name))
    data = data.round(1)

Button {} is pressed stop_dining = 
 True
Button Press Monitor Thread-22 joined
PhilosopherThread-23 joined
PhilosopherThread-24 joined
PhilosopherThread-25 joined
PhilosopherThread-26 joined
PhilosopherThread-27 joined


In [10]:
data

,run time,eat count,nap count,starve count,starving time,eating time,napping time,time sum
Phil 0,32.7,2.0,2.0,24.0,10.2,16.2,4.0,30.4
Phil 1,35.7,2.0,2.0,54.0,15.3,14.1,7.0,36.5
Phil 2,30.7,2.0,2.0,83.0,8.1,14.1,2.0,24.2
Phil 3,32.7,1.0,1.0,91.0,23.2,8.1,2.0,33.3
Phil 4,50.7,2.0,2.0,32.0,13.3,24.3,14.0,51.6
